In [42]:
import preprocessing_data as prep
import pandas as pd
import numpy as np
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', 50)

# Load Season Player Data

All features:   
date,stadium,inning,side,pitcher,pitch_count,batter,balls,strikes,  
ay,px,ax,sz_bot,vz0,vy0,  
pfx_x,type_confidence,z0,tfs,pz,start_speed,  
az,zone,break_angle,spin_dir,end_speed,vx0,  
sz_top,nasty,descr,pfx_z,break_y,pitch_type,  
tfs_zulu,x,spin_rate,y0,break_length,y,x0,  
on_1b,on_2b,on_3b,umpcall,outcome,offense_score,defense_score

In [43]:
years = [2,3,4,5]
base_dir = "Data/"
# features_to_keep = ["pitcher","batter", "balls","strikes","pitch_count","inning","side", "umpcall"]
features_to_load = ["pitcher","batter","pitch_type","start_speed","spin_dir","umpcall"]
train_regular_season = [base_dir+"MLB_201{0}/MLB_PitchFX_201{0}_RegularSeason.csv".format(i) for i in years]
train_post_season = [base_dir+"MLB_201{0}/MLB_PitchFX_201{0}_RegularSeason.csv".format(i) for i in years]
all_data = prep.read_and_combine_data(train_regular_season+train_post_season,features_to_load)

In [44]:
players = pd.read_csv(base_dir+"MLB_Players.csv")

# Basic Statistics

In [45]:
pitchers_name_s = all_data.pitcher.unique()
batters_name_s = all_data.batter.unique()
all_name_s = np.unique(np.append(pitchers_name_s,batters_name_s))
player_stats = {}

In [46]:
pitch_types = ["FA","FF","FT","FC","FS",
               "SI","SF","SL","CH","CB",
               "CU","KC","KN","EP","UN",
               "XX","PO","FO"
              ]
pitch_types = all_data.pitch_type.unique()
def gen_empty_pitch_type(types):
    result = {}
    for t in types:
        result[t] = {
            "count":0,
            "start_speed":0,
            "spin_dir":0,
            "umpcall":{
                'X':0,
                'S':0,
                'B':0
            }
        }
    return result
def new_player_stat():
    result = {"bat_count":0, 
             "pitch_count":0,
             "hit_count":0, 
             "pitch_type":gen_empty_pitch_type(pitch_types),
             "at_bats":0}
    return result

In [47]:
old_batter = ""
init = True
for row in all_data.itertuples(index=False):
#     print(row)
    if not init:
        old_batter = batter
    batter = row.batter
    pitcher = row.pitcher
    
    # do batter first
    if batter not in player_stats:
        player_stats[batter] = new_player_stat()
   
    if old_batter != batter:
        player_stats[batter]["at_bats"] += 1
    player_stats[batter]["bat_count"] += 1
    
    if row.umpcall is 'X':
        player_stats[batter]["hit_count"] +=1
    
    
    
    # do pitcher 
    
    if pitcher not in player_stats:
        player_stats[pitcher] = new_player_stat()
    player_stats[pitcher]["pitch_count"] += 1
    
    p = player_stats[pitcher]["pitch_type"][row.pitch_type]
    p["count"]+=1
    p["start_speed"]+=row.start_speed
    p["spin_dir"]+=row.spin_dir
    p["umpcall"][row.umpcall]+=1
    init = False

In [48]:
# Calculate avg stats
for player,stat in player_stats.items():
    strike_count = 0
    for pitch_type,pitch_type_val in stat["pitch_type"].items():
        if(pitch_type_val["count"]>0):
            pitch_type_val["avg_start_speed"] = pitch_type_val["start_speed"]/pitch_type_val["count"]
            pitch_type_val["avg_spin_dir"] = pitch_type_val["spin_dir"]/pitch_type_val["count"]
            pitch_type_val["possibility"] = pitch_type_val["count"]/stat["pitch_count"]
            strike_count +=pitch_type_val["umpcall"]["S"]
        else:
            pitch_type_val["avg_start_speed"] = 0
            pitch_type_val["avg_spin_dir"] = 0       
            pitch_type_val["possibility"] = 0
    if stat["pitch_count"] > 0:
        stat["strike_ratio"] = strike_count / stat["pitch_count"]
    else:
        stat["strike_ratio"] = 0
        
    if stat["bat_count"] > 0:
        stat["hit_ratio"] = stat["hit_count"] / stat["bat_count"]
    else:
        stat["hit_ratio"] = 0

In [49]:
# for pitcher in pitchers_name_s[:5]:
#     print("\nPlayer: ",pitcher)
#     plyer = player_stats[pitcher]
#     print("Strike ratio: ",plyer["strike_ratio"])
#     for pitch_type,pitch_type_val in plyer["pitch_type"].items():
#         if(pitch_type_val["count"]>0):
#             print(pitch_type,pitch_type_val)

In [50]:

# for batter in batters_name_s[:5]:
#     print("\nPlayer: ",batter)
#     plyer = player_stats[batter]
#     print("hit_ratio", plyer["hit_ratio"])
#     print("hit", plyer["hit_count"])
#     print("bats", plyer["bat_count"])

In [51]:
def new_col_names(pitch_types, rest):
    col = [pth for pth in pitch_types]
    col += [pth+"_speed" for pth in pitch_types]
    col += rest
    return col
for c  in new_col_names(pitch_types, ["hit_ratio"]):
    players[c] = 0 
players["most_likely_pitch"] = "FF"
temp = players.copy()
temp = temp.drop(temp.index[0:])

for index, player in players.iterrows():
    pid = player["bref_id"]
    if pid in player_stats:
        most_like = 0
        plyer = player_stats[pid]
        for pitch_type,pitch_type_val in plyer["pitch_type"].items():
            if(pitch_type_val["count"]>0):
                player[pitch_type] = pitch_type_val["possibility"]
                player[pitch_type+"_speed"] = pitch_type_val["avg_start_speed"]
                if pitch_type_val["possibility"] > most_like:
                    print(pitch_type_val["possibility"])
                    player["most_likely_pitch"] = pitch_type
                    most_like = pitch_type_val["possibility"]
        player["hit_ratio"] = plyer["hit_ratio"]
        player["strike_ratio"] = plyer["strike_ratio"]
    temp = temp.append(player)

0.3116961414790997
0.533862540192926
0.24324324324324326
0.7477477477477478
0.6400771952396269
0.6034563288183092
0.040863890615289
0.25823492852703545
0.3926351771286513
0.3654808959156785
0.2613544201135442
0.5050531914893617
0.31683168316831684
0.48254620123203285
0.3654046182650302
1.0
0.02877959927140255
0.10200364298724955
0.6196721311475409
0.5969604863221885
0.026905829596412557
0.2701345291479821
0.44887892376681615
0.3057324840764331
0.41321656050955413
0.3183098591549296
0.36901408450704226
0.24864864864864866
0.4810810810810811
0.2777777777777778
0.3333333333333333
0.531390134529148
0.28294862248696945
0.16076845298281092
0.442366026289181
0.4400983721747277
0.588909426987061
0.41575225554742745
0.24552845528455283
0.2528455284552846
0.3065040650406504
0.5319148936170213
0.5547874179052886
0.09406175771971496
0.3548693586698337
0.5501187648456057
0.29333804559109383
0.49047031399972574
0.42058230037474775
0.6414201183431952
0.36657303370786515
0.3693820224719101
0.566666666

0.19849812265331665
0.3686274509803922
0.3902107409925221
0.6373626373626373
0.4454492976717337
0.4230284994849491
0.02998965873836608
0.2549120992761117
0.5413650465356774
0.05725190839694656
0.16030534351145037
0.5534351145038168
0.578080229226361
0.11518683579019541
0.612958519026397
0.3076923076923077
0.3784355179704017
0.5306553911205074
0.2747190480687506
0.5327963176064442
0.438449240607514
0.39209726443769
0.49096138044371407
0.5138592750533049
0.3755020080321285
0.5983935742971888
0.6685082872928176
0.4433833560709413
0.3736133122028526
0.47781299524564186
0.3161525974025974
0.11950185479597245
0.1332803391626921
0.5378908320084791
0.37927565392354123
1.0
0.06349206349206349
0.3315696649029982
0.5696649029982364
0.6388888888888888
0.38236977676016026
0.5818385650224215
0.4514480408858603
0.4855195911413969
0.43703703703703706
0.592436974789916
0.15436005625879043
0.32032348804500704
0.4258087201125176
0.008620689655172414
0.31417624521072796
0.60727969348659
0.631974921630094


0.04403774663997712
0.1821561338289963
0.5922219044895625
0.603992446722417
0.2773972602739726
0.46390937829293993
0.3558447937131631
0.3235294117647059
0.47058823529411764
0.49665551839464883
0.5548172757475083
0.5912408759124088
0.6955602536997886
0.3242690410299603
1.0
0.22818791946308725
0.25838926174496646
0.30201342281879195
0.3213754093480202
0.40547782077999406
0.6666666666666666
0.5471014492753623
0.6967213114754098
0.35365853658536583
0.3698630136986301
0.3904109589041096
0.3967611336032389
0.2566478646253022
0.3170829975825947
0.6442953020134228
0.5833333333333334
0.30711444984533803
0.8181818181818182
0.27435265104808876
0.2922318125770654
0.49078341013824883
0.5398948877209747
0.35990338164251207
0.5392857142857143
0.48484848484848486
0.3220640569395018
0.3445017182130584
0.21774193548387097
0.31451612903225806
0.5853084856160666
0.2
0.7647058823529411
0.059569074778200254
0.6337135614702155
0.27155802396959955
0.2744811458637825
0.337000366703337
0.5162654521795705
0.3879

0.768012263668881
0.6218074656188605
0.35787172011661805
0.25125628140703515
0.41708542713567837
0.3713646532438479
0.39149888143176736
0.6305732484076433
0.19487179487179487
0.22564102564102564
0.2512820512820513
0.578760162601626
0.35833333333333334
0.5059523809523809
0.4424778761061947
0.4932975871313673
0.44666666666666666
0.5425531914893617
0.40331491712707185
0.5646481178396072
0.15058365758754863
0.28949416342412454
0.3167315175097276
0.08686440677966102
0.125
0.5190677966101694
0.4943181818181818
0.16810172206647978
0.17851421706047257
0.41159391269523427
0.40297450424929177
0.2665410878267012
0.32093242288674356
0.5052954292084727
0.5279912184412733
0.5040221216691805
0.3484912877178071
0.3560500695410292
0.2132701421800948
0.2843601895734597
0.2938388625592417
0.5245901639344263
0.43376623376623374
0.6665770368378596
0.6633761105626851
0.01834862385321101
0.2889908256880734
0.5
0.1776765375854214
0.42596810933940776
0.2096128170894526
0.7069425901201603
0.10597826086956522
0.

In [52]:
# print(players[:5])

In [53]:
temp[:5]


,mlb_id,bref_id,last,first,throws,bats,height,weight,dob,FF,SL,SI,CH,FT,CU,KC,FC,FS,FF_speed,SL_speed,SI_speed,CH_speed,FT_speed,CU_speed,KC_speed,FC_speed,FS_speed,hit_ratio,most_likely_pitch,strike_ratio
0,112526,colonba01,Colon,Bartolo,R,R,71.0,265.0,1973-05-24,0.311696,0.099176,0.0,0.055265,0.533863,0.0,0.0,0.0,0.0,90.97724,82.546201,0.0,81.813455,88.151647,0.0,0.0,0.0,0.0,0.184019,FT,0.465535
1,134181,beltrad01,Beltre,Adrian,R,R,71.0,220.0,1979-04-07,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.220411,FF,0.000000
2,136860,beltrca01,Beltran,Carlos,R,S,73.0,215.0,1977-04-24,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.197482,FF,0.000000
3,150029,werthja01,Werth,Jayson,R,R,77.0,220.0,1979-05-20,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.162191,FF,0.000000
4,150212,cuddymi01,Cuddyer,Michael,R,NaN,74.0,220.0,1979-03-27,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.191378,FF,0.000000


In [54]:
# player_stats["beltrad01"]

In [55]:
temp.to_csv("MLB_Players_Stats.csv" )